In [46]:
import geopandas 
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
from shapely.geometry import Point, Polygon, LineString
import seaborn as sns 
sns.set()
import descartes
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians
import numpy as np

lakes = geopandas.read_file("Lakes_and_Rivers")
madison = geopandas.read_file("City_Limit")

In [47]:
#https://data-cityofmadison.opendata.arcgis.com/datasets/metro-transit-ridership-by-route-weekday?geometry=-89.671%2C42.985%2C-89.23%2C43.161
weekday_bus = geopandas.read_file("Metro_Transit_Ridership_by_Route_Weekday")



In [48]:
#metro data
metro_data = geopandas.read_file("MetroBusRoutes.gdb")
list_metro = []
for index, row in metro_data.iterrows():
    list_metro.append(Point(float(row["stop_lon"]),float(row["stop_lat"])))

metro_series = pd.Series(list_metro)
metro_coord = pd.DataFrame({"Coordinates": metro_series})
metro_coord = geopandas.GeoDataFrame(metro_coord, geometry='Coordinates')

#BRT data 
#https://data-cityofmadison.opendata.arcgis.com/datasets/bus-rapid-transit-stops
# using new data for brt from the madison data instead of the BRT.gdb
brt_data = geopandas.read_file("Bus_Rapid_Transit_Stops")
list_brt = []
for index, row in brt_data.iterrows():
    list_brt.append(Point(float(row["stop_lon"]),float(row["stop_lat"])))

brt_series = pd.Series(list_brt)
brt_coord = pd.DataFrame({"Coordinates": brt_series})
brt_coord = geopandas.GeoDataFrame(brt_coord, geometry='Coordinates')



In [49]:
#ref _ https://stackoverflow.com/questions/10202570/find-row-where-values-for-column-is-maximal-in-a-pandas-dataframe
# ref _ http://postgis.org/docs/ST_Distance.html
weekday_bus['brt_index'] = ''
weekday_bus['distance_to_brt'] = ''

for index, row in weekday_bus.iterrows():
    list_bus = []
    for index1, row1 in brt_data.iterrows(): 
        dist = row['geometry'].distance(row1['geometry'])
        list_bus.append(dist)
    list_bus_df = pd.DataFrame(list_bus)
    list_bus_df.columns =['distance_from_nearest_brt']

    index_of_brt = list_bus_df['distance_from_nearest_brt'].idxmin()
    distance_to_nearest_brt = list_bus_df['distance_from_nearest_brt'].min()

    weekday_bus.at[index, 'brt_index'] = index_of_brt
    weekday_bus.at[index, 'distance_to_brt'] = distance_to_nearest_brt

In [50]:
weekday_bus

,OBJECTID,StopID,Route,Lat,Lon,DailyBoard,DotSize,geometry,brt_index,distance_to_brt
0,16341,4100,4,43.038895,-89.394824,189.84,1898447.45,POINT (-89.39482 43.03889),59,0.000143962
1,16342,4100,5,43.038895,-89.394824,210.71,2107109.73,POINT (-89.39482 43.03889),59,0.000143962
2,16343,4100,13,43.038895,-89.394824,79.50,795000.00,POINT (-89.39482 43.03889),59,0.000143962
3,16344,4100,16,43.038895,-89.394824,185.31,1853106.79,POINT (-89.39482 43.03889),59,0.000143962
4,16345,4100,18,43.038895,-89.394824,299.38,2993805.57,POINT (-89.39482 43.03889),59,0.000143962
5,16346,4100,40,43.038895,-89.394824,258.62,2586153.85,POINT (-89.39482 43.03889),59,0.000143962
6,16347,4100,44,43.038895,-89.394824,18.51,185067.42,POINT (-89.39482 43.03889),59,0.000143962
7,16348,4100,48,43.038895,-89.394824,5.67,56666.67,POINT (-89.39482 43.03889),59,0.000143962
8,16349,4100,49,43.038895,-89.394824,13.83,138333.33,POINT (-89.39482 43.03889),59,0.000143962
9,16350,5100,2,43.113253,-89.358540,192.14,1921423.01,POINT (-89.35853 43.11325),116,0.00210804


In [76]:
numReds = weekday_bus[weekday_bus["distance_to_brt"]<=0.0013]

totalStops = len(weekday_bus)

#5 minute walk: 0.0013 degrees
#from BRT_Metro_maps_and_graphs.ipynb


print(len(numReds), totalStops)

639 3666


In [87]:
639 * 2500

1597500

In [89]:
13343 * 10

133430

In [77]:
redundancies = list(weekday_bus[weekday_bus["distance_to_brt"]<=0.0013]["OBJECTID"])

In [86]:
#https://data-cityofmadison.opendata.arcgis.com/datasets/metro-transit-ridership-by-route-weekday?geometry=-89.671%2C42.985%2C-89.23%2C43.161
ridership = geopandas.read_file("Metro_Transit_Ridership_by_Route_Weekday")

total_ridership = 0
ridership["DailyBoard"][0]
for stop in redundancies:
    boarding = list(ridership[ridership["OBJECTID"]==stop]["DailyBoard"])[0]
    total_ridership+=boarding
aveRedRider = total_ridership/len(numReds)
total_ridership

13343.390000000036

In [83]:
outsideBRT = list(weekday_bus[weekday_bus["distance_to_brt"]>0.0013]["OBJECTID"])
print(len(outsideBRT), totalStops)

3027 3666


In [84]:
outsideRiders = 0
for stop in outsideBRT:
    boarding = list(ridership[ridership["OBJECTID"]==stop]["DailyBoard"])[0]
    total_ridership+=boarding
aveOutsideRider = total_ridership/len(outsideBRT)

In [85]:
#PERCENT CHANGE WITH AVERAGE RIDERSHIP
(aveRedRider-aveOutsideRider)/aveRedRider

0.16756707734858134

In [41]:
# weekday_bus.to_csv('weekday_bus_with_brt_distance.gpkg', encoding='utf-8', index=False)

In [7]:
weekday_bus.to_file("weekday_bus_with_brt_distance.gpkg", driver="GPKG")

ValueError: Invalid field type <class 'numpy.int64'>